Convolutional Neural Networks for MNIST dataset

Configure training flags

In [1]:
import os
import sys

nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import torch
import matplotlib.pyplot as plt
import numpy as np

from cnn.mnist.cnn_files import files as _files

plt.ion()

def init_training_flags():
    
    tr_flags = type('TrainingFlags', (object,), {})
    
    tr_flags.batch_size = 64
    tr_flags.test_batch_size = 1000
    tr_flags.epochs = 10
    tr_flags.no_cuda = False
    tr_flags.seed = 1
    tr_flags.log_interval = 10
    tr_flags.weights = _files.model_file('mnist_weights.pth.tar')
    tr_flags.lr = 0.01
    tr_flags.momentum = 0.5
    
    # System configuration
    tr_flags.num_workers=8
    
    tr_flags.cuda = False

    return tr_flags
    


Prepare datasets

In [3]:
from torch import optim
from torch.autograd import Variable
from torchvision import (datasets, transforms)

flags = init_training_flags()

torch.manual_seed(flags.seed)
if flags.cuda:
  torch.cuda.manual_seed(flags.seed)

kwargs = {'num_workers': flags.num_workers, 'pin_memory': True} if flags.cuda else {}
train_loader = torch.utils.data.DataLoader(
  datasets.MNIST(_files.data_dir, train=True, download=True,
                 transform=transforms.Compose([
                     transforms.ToTensor(),
                     transforms.Normalize((0.1307,), (0.3081,))
                 ])),
  batch_size=flags.batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
  datasets.MNIST(_files.data_dir, train=False, transform=transforms.Compose([
                     transforms.ToTensor(),
                     transforms.Normalize((0.1307,), (0.3081,))
                 ])),
  batch_size=flags.test_batch_size, shuffle=True, **kwargs)

Initialize model

In [11]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from collections import OrderedDict

from torch import nn

import torch.nn.functional as F

from utils.models.layers import Flatten


class LeNetClassic(nn.Module):
  """Network model without flatten layer
   for character recognition"""
  
  def __init__(self):
    super(LeNetClassic, self).__init__()
    self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
    self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
    self.conv2_drop = nn.Dropout2d()
    self.fc1 = nn.Linear(320, 50)
    self.fc2 = nn.Linear(50, 10)
  
  def forward(self, x):
      
    x = F.relu(F.max_pool2d(self.conv1(x), 2))
    x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
    x = x.view(x.size(0), 320)
    x = F.relu(self.fc1(x))
    x = F.dropout(x, training=self.training)
    x = self.fc2(x)
    result = F.log_softmax(x)
    
    return result


class LeNet(nn.Module):
  """Network model with flatten layer
   for character recognition"""
  
  def __init__(self):
    super(LeNet, self).__init__()
    self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
    self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
    self.conv2_drop = nn.Dropout2d()
    self.flatten = Flatten(50)
    self.fc2 = nn.Linear(50, 10)
  
  def forward(self, x):
      
    x = F.relu(F.max_pool2d(self.conv1(x), 2))
    x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
    x = self.flatten(x)
    x = F.relu(x)
    x = F.dropout(x, training=self.training)
    x = self.fc2(x)
    result = F.log_softmax(x)
    
    return result


class LeNetSequential(nn.Module):
  """Network model with flatten layer
   for character recognition"""
  
  def __init__(self):
    super(LeNetSequential, self).__init__()
    self.conv_part = nn.Sequential(nn.Conv2d(1, 10, kernel_size=5),
                                   nn.MaxPool2d(2, 2),
                                   nn.ReLU(),
                                   nn.Conv2d(10, 20, kernel_size=5),
                                   nn.MaxPool2d(2, 2),
                                   nn.ReLU(),
                                   nn.Dropout2d())
    self.flatten = Flatten(50)
    self.fc2 = nn.Linear(50, 10)
  
  def forward(self, x):
      
    x = self.conv_part(x)
    x = self.flatten(x)
    x = F.relu(x)
    x = F.dropout(x, training=self.training)
    x = self.fc2(x)
    result = F.log_softmax(x)
    
    return result
  
  
class LeNetSequentialDict(nn.Module):
  """Network model with flatten layer
   for character recognition"""
  
  def __init__(self):
    super(LeNetSequentialDict, self).__init__()
    self.conv_part = nn.Sequential(OrderedDict([
                                   ('conv1', nn.Conv2d(1, 10, kernel_size=5)),
                                   ('mxpl1', nn.MaxPool2d(2, 2)),
                                   ('relu1', nn.ReLU()),
                                   ('conv2', nn.Conv2d(10, 20, kernel_size=5)),
                                   ('mxol2', nn.MaxPool2d(2, 2)),
                                   ('relu2', nn.ReLU()),
                                   ('drop1', nn.Dropout2d())]))
    self.flatten = Flatten(50)
    self.fc2 = nn.Linear(50, 10)
  
  def forward(self, x):
      
    x = self.conv_part(x)
    x = self.flatten(x)
    x = F.relu(x)
    x = F.dropout(x, training=self.training)
    x = self.fc2(x)
    result = F.log_softmax(x)
    
    return result



Traing the network

In [10]:
def train(epoch, training_config):
  """Train network model
    Args:
      epoch - current epoch
      training_config - training configuration tuple
  """
    
  (train_loader, model, optimizer, flags) = training_config
  model.train()
  for (batch_idx, (data, target)) in enumerate(train_loader):
    if flags.cuda:
        (data, target) = (data.cuda(), target.cusa())
    (data, target) = (Variable(data), Variable(target))
    optimizer.zero_grad()
    output = model(data)
    loss = F.nll_loss(output, target)
    loss.backward()
    optimizer.step()
    if batch_idx % flags.log_interval == 0:
      print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
          epoch, batch_idx * len(data), len(train_loader.dataset),
          100. * batch_idx / len(train_loader), loss.data[0]))
  torch.save(model.state_dict(), flags.weights)
  

def test(test_loader, model, flags):
  """Test network
    test_loader - test data loader
    model - network model
    flags - configuration parameters
  """
    
  model.eval()
  test_loss = 0
  correct = 0
  for (data, target) in test_loader:
    if flags.cuda:
        (data, target) = (data.cuda(), target.cusa())
    (data, target) = (Variable(data, volatile=True), Variable(target))
    output = model(data)
    test_loss += F.nll_loss(output, target, size_average=False).data[0]  # sum up batch loss
    pred = output.data.max(1, keepdim=True)[1]  # get the index of the max log-probability
    correct += pred.eq(target.data.view_as(pred)).cpu().sum()

  test_loss /= len(test_loader.dataset)
  print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
      test_loss, correct, len(test_loader.dataset),
      100. * correct / len(test_loader.dataset)))

model = LetterNet()

if flags.cuda:
    model.cuda()

print('End=attachment')
optimizer = optim.SGD(model.parameters(), lr=flags.lr, momentum=flags.momentum)
training_config = (train_loader, model, optimizer, flags)
for epoch in range(1, flags.epochs + 1):
    train(epoch, training_config)
    test(test_loader, model, flags)

End=attachment
Train Epoch: 1 [0/60000 (0%)]	Loss: 2.291420
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.320766
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.293607
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.262021
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.299063
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.253126
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.246339
Train Epoch: 1 [4480/60000 (7%)]	Loss: 2.265384
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.266011
Train Epoch: 1 [5760/60000 (10%)]	Loss: 2.283992
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.210891
Train Epoch: 1 [7040/60000 (12%)]	Loss: 2.295862
Train Epoch: 1 [7680/60000 (13%)]	Loss: 2.198273
Train Epoch: 1 [8320/60000 (14%)]	Loss: 2.182696
Train Epoch: 1 [8960/60000 (15%)]	Loss: 2.153505
Train Epoch: 1 [9600/60000 (16%)]	Loss: 2.201344
Train Epoch: 1 [10240/60000 (17%)]	Loss: 2.230200
Train Epoch: 1 [10880/60000 (18%)]	Loss: 2.240868
Train Epoch: 1 [11520/60000 (19%)]	Loss: 2.217868
Train Epoch: 1 [12160/60000 (20%)]	Loss: 2.112736
Train Epoch: 1

Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.823559
Train Epoch: 2 [45440/60000 (76%)]	Loss: 1.031581
Train Epoch: 2 [46080/60000 (77%)]	Loss: 1.069610
Train Epoch: 2 [46720/60000 (78%)]	Loss: 0.992646
Train Epoch: 2 [47360/60000 (79%)]	Loss: 1.070144
Train Epoch: 2 [48000/60000 (80%)]	Loss: 1.196937
Train Epoch: 2 [48640/60000 (81%)]	Loss: 1.013552
Train Epoch: 2 [49280/60000 (82%)]	Loss: 1.007919
Train Epoch: 2 [49920/60000 (83%)]	Loss: 1.403352
Train Epoch: 2 [50560/60000 (84%)]	Loss: 1.017321
Train Epoch: 2 [51200/60000 (85%)]	Loss: 1.067889
Train Epoch: 2 [51840/60000 (86%)]	Loss: 1.226116
Train Epoch: 2 [52480/60000 (87%)]	Loss: 0.975467
Train Epoch: 2 [53120/60000 (88%)]	Loss: 1.086594
Train Epoch: 2 [53760/60000 (90%)]	Loss: 1.060130
Train Epoch: 2 [54400/60000 (91%)]	Loss: 1.025473
Train Epoch: 2 [55040/60000 (92%)]	Loss: 1.512197
Train Epoch: 2 [55680/60000 (93%)]	Loss: 1.115833
Train Epoch: 2 [56320/60000 (94%)]	Loss: 0.938655
Train Epoch: 2 [56960/60000 (95%)]	Loss: 1.328765


Train Epoch: 4 [28800/60000 (48%)]	Loss: 0.580717
Train Epoch: 4 [29440/60000 (49%)]	Loss: 0.886387
Train Epoch: 4 [30080/60000 (50%)]	Loss: 1.135479
Train Epoch: 4 [30720/60000 (51%)]	Loss: 0.959838
Train Epoch: 4 [31360/60000 (52%)]	Loss: 0.959866
Train Epoch: 4 [32000/60000 (53%)]	Loss: 0.949440
Train Epoch: 4 [32640/60000 (54%)]	Loss: 0.977598
Train Epoch: 4 [33280/60000 (55%)]	Loss: 0.698675
Train Epoch: 4 [33920/60000 (57%)]	Loss: 1.198358
Train Epoch: 4 [34560/60000 (58%)]	Loss: 0.887278
Train Epoch: 4 [35200/60000 (59%)]	Loss: 0.838312
Train Epoch: 4 [35840/60000 (60%)]	Loss: 1.095731
Train Epoch: 4 [36480/60000 (61%)]	Loss: 1.114238
Train Epoch: 4 [37120/60000 (62%)]	Loss: 0.752053
Train Epoch: 4 [37760/60000 (63%)]	Loss: 0.736348
Train Epoch: 4 [38400/60000 (64%)]	Loss: 1.205590
Train Epoch: 4 [39040/60000 (65%)]	Loss: 1.154064
Train Epoch: 4 [39680/60000 (66%)]	Loss: 1.028600
Train Epoch: 4 [40320/60000 (67%)]	Loss: 1.042127
Train Epoch: 4 [40960/60000 (68%)]	Loss: 0.988634


Train Epoch: 6 [12800/60000 (21%)]	Loss: 0.797495
Train Epoch: 6 [13440/60000 (22%)]	Loss: 0.788341
Train Epoch: 6 [14080/60000 (23%)]	Loss: 0.698124
Train Epoch: 6 [14720/60000 (25%)]	Loss: 0.821524
Train Epoch: 6 [15360/60000 (26%)]	Loss: 0.977970
Train Epoch: 6 [16000/60000 (27%)]	Loss: 0.684955
Train Epoch: 6 [16640/60000 (28%)]	Loss: 1.049334
Train Epoch: 6 [17280/60000 (29%)]	Loss: 0.724926
Train Epoch: 6 [17920/60000 (30%)]	Loss: 0.956910
Train Epoch: 6 [18560/60000 (31%)]	Loss: 0.863667
Train Epoch: 6 [19200/60000 (32%)]	Loss: 0.766245
Train Epoch: 6 [19840/60000 (33%)]	Loss: 0.853020
Train Epoch: 6 [20480/60000 (34%)]	Loss: 0.584582
Train Epoch: 6 [21120/60000 (35%)]	Loss: 0.871847
Train Epoch: 6 [21760/60000 (36%)]	Loss: 0.747976
Train Epoch: 6 [22400/60000 (37%)]	Loss: 0.954876
Train Epoch: 6 [23040/60000 (38%)]	Loss: 0.787820
Train Epoch: 6 [23680/60000 (39%)]	Loss: 0.913367
Train Epoch: 6 [24320/60000 (41%)]	Loss: 0.950986
Train Epoch: 6 [24960/60000 (42%)]	Loss: 1.040218


Train Epoch: 7 [57600/60000 (96%)]	Loss: 0.557062
Train Epoch: 7 [58240/60000 (97%)]	Loss: 0.694381
Train Epoch: 7 [58880/60000 (98%)]	Loss: 0.744915
Train Epoch: 7 [59520/60000 (99%)]	Loss: 0.905197

Test set: Average loss: 0.2659, Accuracy: 9486/10000 (95%)

Train Epoch: 8 [0/60000 (0%)]	Loss: 0.750619
Train Epoch: 8 [640/60000 (1%)]	Loss: 0.650739
Train Epoch: 8 [1280/60000 (2%)]	Loss: 0.915138
Train Epoch: 8 [1920/60000 (3%)]	Loss: 0.974229
Train Epoch: 8 [2560/60000 (4%)]	Loss: 0.713361
Train Epoch: 8 [3200/60000 (5%)]	Loss: 0.868833
Train Epoch: 8 [3840/60000 (6%)]	Loss: 0.971016
Train Epoch: 8 [4480/60000 (7%)]	Loss: 0.944800
Train Epoch: 8 [5120/60000 (9%)]	Loss: 0.804861
Train Epoch: 8 [5760/60000 (10%)]	Loss: 0.821842
Train Epoch: 8 [6400/60000 (11%)]	Loss: 0.728135
Train Epoch: 8 [7040/60000 (12%)]	Loss: 0.764291
Train Epoch: 8 [7680/60000 (13%)]	Loss: 0.881551
Train Epoch: 8 [8320/60000 (14%)]	Loss: 0.890386
Train Epoch: 8 [8960/60000 (15%)]	Loss: 0.736806
Train Epoch: 8 [9

Train Epoch: 9 [41600/60000 (69%)]	Loss: 0.757240
Train Epoch: 9 [42240/60000 (70%)]	Loss: 0.826704
Train Epoch: 9 [42880/60000 (71%)]	Loss: 0.780100
Train Epoch: 9 [43520/60000 (72%)]	Loss: 0.795745
Train Epoch: 9 [44160/60000 (74%)]	Loss: 0.509552
Train Epoch: 9 [44800/60000 (75%)]	Loss: 0.683279
Train Epoch: 9 [45440/60000 (76%)]	Loss: 0.822753
Train Epoch: 9 [46080/60000 (77%)]	Loss: 0.681254
Train Epoch: 9 [46720/60000 (78%)]	Loss: 0.691773
Train Epoch: 9 [47360/60000 (79%)]	Loss: 0.993729
Train Epoch: 9 [48000/60000 (80%)]	Loss: 0.659368
Train Epoch: 9 [48640/60000 (81%)]	Loss: 1.005893
Train Epoch: 9 [49280/60000 (82%)]	Loss: 0.821565
Train Epoch: 9 [49920/60000 (83%)]	Loss: 0.856251
Train Epoch: 9 [50560/60000 (84%)]	Loss: 0.477131
Train Epoch: 9 [51200/60000 (85%)]	Loss: 0.472896
Train Epoch: 9 [51840/60000 (86%)]	Loss: 0.712545
Train Epoch: 9 [52480/60000 (87%)]	Loss: 1.002561
Train Epoch: 9 [53120/60000 (88%)]	Loss: 0.750750
Train Epoch: 9 [53760/60000 (90%)]	Loss: 0.809389
